In [158]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
from scipy.spatial.distance import euclidean

In [159]:
data = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
data.dropna(inplace=True)

/home/eketerina753/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [160]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [161]:
X=data.iloc[:100000,3:5]
ms=MeanShift(bandwidth=0.1).fit(X)

In [237]:
centers=ms.cluster_centers_
labels=ms.labels_

print(centers)

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [ -37.8229826   145.1811902 ]
 [ -41.2924945   174.7732353 ]
 [ -45.0311622   168.6626435 ]]


In [163]:
# считаем колическтво объектов для каждого кластера, где индекс в массиве - номер кластера
count_in_clusters=np.bincount(labels)

In [164]:
print(count_in_clusters)

[12506  4692  3994 ...     1     1     1]


In [239]:
#если кол-во объектов в кластере меньше 16, то заменяем его на 0, чтобы потом с помощью функции np.nonzero получить номера кластеров, объектов в которых больше 15
for i in range(len(count_in_clusters)):
    if count_in_clusters[i]<=15:
        count_in_clusters[i]=0
print(count_in_clusters)
actual_index_of_clusters=np.nonzero(count_in_clusters)
#или
#val, counts = np.unique(labels, return_counts=True)
#labels_new = [l for l in val if counts[l] > 15]

[12506  4692  3994 ...     0     0     0]


In [166]:
actual_index_of_clusters

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
          22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
          33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
          44,   45,   46,   47,   48,   49,   50,   51,   52,   53,   54,
          55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
          66,   67,   68,   69,   70,   71,   72,   73,   74,   75,   76,
          77,   78,   79,   80,   81,   82,   83,   84,   85,   86,   87,
          88,   89,   90,   91,   92,   93,   94,   95,   96,   97,   98,
          99,  100,  101,  102,  103,  104,  105,  106,  107,  108,  109,
         110,  111,  112,  113,  114,  115,  116,  117,  118,  119,  120,
         121,  122,  123,  124,  125,  126,  127,  128,  129,  130,  131,
         132,  133,  134,  135,  136,  137,  138,  139,  140,  141,  142,
         143,  144,  145,  146,  147, 

In [167]:
# координаты офисов
coords=np.array([[33.751277, -118.188740],

[25.867736, -80.324116],

[51.503016, -0.075479],

[52.378894, 4.885084],

[39.366487, 117.036146],

[-33.868457, 151.205134]])

In [168]:
# получаем координаты центров кластеров, в которых больше 15 объектов
actual_centers=centers[actual_index_of_clusters]
print(centers[1])

[  33.44943805 -112.00213969]


In [228]:
#считаем евклидово расстояние между каждым офисом и центром каждого класреа
dist=[]
for center in actual_centers:
    for i in range(len(coords)):
        dist.append(euclidean(coords[i,:], center))

In [230]:
len(dist)
d=dist

In [231]:
# 20 раз ищем минимум, присваивая очередному минимуму макс значение, чтобы оно повторно не добавлялось в массив шндексв минимумов
min_center_index=[]
maximum=max(dist)
print(dist[2507])
for i in range(20):
    ind=0
    for i in range(1,len(d)):
        if d[i]<d[ind]:
            ind=i
    print(ind)
    d[ind]=maximum
    min_center_index.append(ind)
            
        

0.007834758163107856
2507
2217
2497
350
306
175
1003
553
522
252
1746
1921
714
331
162
66
192
955
102
282


In [232]:
#вычиляем индекс клстера с учетом того, для для каждого кластера расстояние считалось 6 раз и они записаны подряд для каждого класса
print(min_center_index)
for i in range(len(min_center_index)):
    min_center_index[i]//=6


[2507, 2217, 2497, 350, 306, 175, 1003, 553, 522, 252, 1746, 1921, 714, 331, 162, 66, 192, 955, 102, 282]


In [233]:
min_center_index

[417,
 369,
 416,
 58,
 51,
 29,
 167,
 92,
 87,
 42,
 291,
 320,
 119,
 55,
 27,
 11,
 32,
 159,
 17,
 47]

In [235]:
#наименьшее расстояние имеет первый элемент из этого списка
actual_centers[min_center_index[0]]

array([-33.86063043, 151.20477593])

In [236]:
f=open('ans.txt','w')
f.write('-33.86063043 151.20477593')
f.close()
!cat ans.txt

-33.86063043 151.20477593